# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
fields = {'BusinessName': 1, 'scores.Hygiene': 1}

# Use count_documents to display the number of documents in the result
document_count = establishments.count_documents(query)
print("Total document count:", document_count)

# Display the first document in the results using pprint
results = establishments.find(query, fields)
pprint(results[0])

Total document count: 41
{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('669caf5bd2052717a4e49043'),
 'scores': {'Hygiene': 20}}


In [7]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))

# Display the first 10 rows of the DataFrame
result_df['HygieneScore'] = result_df['scores'].apply(lambda x: x['Hygiene'])
result_df.drop(columns=['scores'], inplace=True)
result_df.head(10)

Rows in DataFrame:  41


,_id,BusinessName,HygieneScore
0,669caf5bd2052717a4e49043,The Chase Rest Home,20
1,669caf5bd2052717a4e493c6,Brenalwood,20
2,669caf5bd2052717a4e496d0,Melrose Hotel,20
3,669caf5bd2052717a4e498c1,Seaford Pizza,20
4,669caf5bd2052717a4e498ca,Golden Palace,20
5,669caf5cd2052717a4e4a26a,Ashby's Butchers,20
6,669caf5cd2052717a4e4a48b,South Sea Express Cuisine,20
7,669caf5cd2052717a4e4b9b8,Golden Palace,20
8,669caf5cd2052717a4e4bdfa,The Tulip Tree,20
9,669caf5cd2052717a4e4c60c,F & S,20


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {'$regex': "London"},
    "RatingValue": {'$gte': 4}}
fields = {'LocalAuthorityName': 1, 'RatingValue': 1}

# Use count_documents to display the number of documents in the result
document_count = establishments.count_documents(query)
print("Total document count:", document_count)

# Display the first document in the results using pprint
results = establishments.find(query, fields)
pprint(results[0])

Total document count: 33
{'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': 4,
 '_id': ObjectId('669caf5cd2052717a4e4aa61')}


In [9]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))

# Display the first 10 rows of the DataFrame
result_df.head(10)

Rows in DataFrame:  33


,_id,RatingValue,LocalAuthorityName
0,669caf5cd2052717a4e4aa61,4,City of London Corporation
1,669caf5cd2052717a4e4ad86,5,City of London Corporation
2,669caf5cd2052717a4e4b8d0,4,City of London Corporation
3,669caf5cd2052717a4e4c6d2,5,City of London Corporation
4,669caf5cd2052717a4e4c6d3,5,City of London Corporation
5,669caf5cd2052717a4e4cdd9,5,City of London Corporation
6,669caf5dd2052717a4e4eee8,5,City of London Corporation
7,669caf5dd2052717a4e4f2b1,4,City of London Corporation
8,669caf5dd2052717a4e4f2cb,5,City of London Corporation
9,669caf5dd2052717a4e4f2cc,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Find the coordinates of "Penang Flavours"
penang_flavors = establishments.find_one({"BusinessName": "Penang Flavours"}, {"geocode.latitude": 1, "geocode.longitude": 1})
degree_search = 0.01
latitude = penang_flavors["geocode"]["latitude"]
longitude = penang_flavors["geocode"]["longitude"]


# Rating value must equal 5
# Search within 0.01 degree on either side of the latitude and longitude.
query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}

# Sort by hygiene score
sort = [("scores.Hygiene", 1)]

# Display top 5 
limit = 5

# Print the results
results = list(establishments.find(query, {"BusinessName": 1, "RatingValue": 1, "scores.Hygiene": 1}).sort(sort).limit(limit))
for result in results:
    print(result)

{'_id': ObjectId('669caf5dd2052717a4e4e88f'), 'BusinessName': 'Iceland', 'RatingValue': 5, 'scores': {'Hygiene': 0}}
{'_id': ObjectId('669caf5dd2052717a4e4e8be'), 'BusinessName': 'Atlantic Fish Bar', 'RatingValue': 5, 'scores': {'Hygiene': 0}}
{'_id': ObjectId('669caf5dd2052717a4e4e8b5'), 'BusinessName': 'Plumstead Manor Nursery', 'RatingValue': 5, 'scores': {'Hygiene': 0}}
{'_id': ObjectId('669caf5dd2052717a4e4e885'), 'BusinessName': 'Howe and Co Fish and Chips - Van 17', 'RatingValue': 5, 'scores': {'Hygiene': 0}}
{'_id': ObjectId('669caf5dd2052717a4e4e8a6'), 'BusinessName': 'Volunteer', 'RatingValue': 5, 'scores': {'Hygiene': 0}}


In [11]:
# Convert result to Pandas DataFrame
result_df = pd.DataFrame(results)
result_df['HygieneScore'] = result_df['scores'].apply(lambda x: x['Hygiene'])
result_df.drop(columns=['scores'], inplace=True)
result_df

,_id,BusinessName,RatingValue,HygieneScore
0,669caf5dd2052717a4e4e88f,Iceland,5,0
1,669caf5dd2052717a4e4e8be,Atlantic Fish Bar,5,0
2,669caf5dd2052717a4e4e8b5,Plumstead Manor Nursery,5,0
3,669caf5dd2052717a4e4e885,Howe and Co Fish and Chips - Van 17,5,0
4,669caf5dd2052717a4e4e8a6,Volunteer,5,0


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [12]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {"$match": {"scores.Hygiene": 0}}

# 2. Groups the matches by Local Authority
group_query = {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}}

# 3. Sorts the matches from highest to lowest
sort_values = {"$sort": {"count": -1, "_id": 1}}

# Print the number of documents in the result
pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))
print("Number of documents in the result:", len(results))

# Print the first 10 results
pprint(results[0:10])

Number of documents in the result: 55
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [13]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))

# Display the first 10 rows of the DataFrame
result_df.head(10)

Rows in DataFrame:  55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
